In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

# Load the dataset
dataset = load_dataset('spider', split='train')

# Initialize the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Preprocessing the data
def preprocess_data(examples):
    inputs = examples['question']
    targets = examples['query']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length').input_ids
    model_inputs['labels'] = labels
    return model_inputs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:

# Tokenize and preprocess the data
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="steps",
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
# Split the dataset into train and validation
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
)

# Fine-tune the model
trainer.train()

Step,Training Loss,Validation Loss
500,0.156100,0.107448
1000,0.125700,0.091463
1500,0.112200,0.082924
2000,0.102200,0.077546
2500,0.095500,0.074127


Step,Training Loss,Validation Loss
500,0.156100,0.107448
1000,0.125700,0.091463
1500,0.112200,0.082924
2000,0.102200,0.077546
2500,0.095500,0.074127
3000,0.092600,0.071926
3500,0.089100,0.070355
4000,0.088900,0.069787


TrainOutput(global_step=4200, training_loss=0.10698290416172572, metrics={'train_runtime': 1827.6388, 'train_samples_per_second': 9.192, 'train_steps_per_second': 2.298, 'total_flos': 2273742264729600.0, 'train_loss': 0.10698290416172572, 'epoch': 3.0})

In [13]:
import torch

def generate_sql_query(natural_language_input):
    inputs = tokenizer(natural_language_input, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    output_sequences = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512)
    return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

# Example usage
nl_input = "List all employees who have a salary greater than 50000."
sql_query = generate_sql_query(nl_input)
print("The Question:", nl_input)
print("Generated SQL Query:", sql_query)

# Example usage 2
nl_input = "Find the names of customers who have made more than 5 purchases."
sql_query = generate_sql_query(nl_input)
print("The Question:", nl_input)
print("Generated SQL Query:", sql_query)

The Question: List all employees who have a salary greater than 50000.
Generated SQL Query: SELECT t1.salari_id FROM Employees AS t1 JOIN salary AS t2 ON t1.salari_id = t3.salari_id WHERE t2.salari_id  50000
The Question: Find the names of customers who have made more than 5 purchases.
Generated SQL Query: SELECT T1.Name FROM Customers WHERE T1.Call_Against_Against_Against_Against > 5
